In [1]:
import pandas as pd

In [2]:
productDF = pd.read_csv('../data/lazada/product.csv')
reviewDF = pd.read_csv('../data/lazada/review.csv')

In [3]:
# extract anonymous review
nonAnonymousReviewDF = reviewDF[reviewDF['user_fk'] != 0]
nonAnonymousReviewDF.sample(5)

,comment,rating,sku,time,user,user_fk
123883,ส่งของสองวันสินค้าดีไม่มีตำหนิสินค้าสวยดีลาซาด...,5,XI482ELAAM0E40ANTH,2017-10-02 09:32:35,Tontan Ritthichai,4009853
28976,ที่แป้นไม่มีภาษาไทย ราคาถูกก็จริง แต่ต้องไปซื้...,3,BO085ELAA4Z0MMANTH,2016-12-26 22:27:10,Manohharn N.,7504256
21359,ใส่ซิมได้ไหมคับรองรับ 4 g ไหม,1,LE611ELAA6KPQHANTH,2017-07-20 20:24:04,ทุกอย่างไม่แน่นอน ปล่อยวาง และปล่อยวาง,1164108
75893,NaN,5,SI803HBAA6WIM1ANTH,2017-04-11 10:29:46,เหมียว,9124064
117589,ส่งได้เร็วตามที่กำหมด<br />\r\nคุณภาพก็ตามราคานะ,4,BL516ELAA3FDLLANTH,2017-04-06 14:30:58,นรินทร์ บ.,502976


In [4]:
# select only data complementing each other
productWithReviewDF = productDF[productDF['sku'].isin(nonAnonymousReviewDF['sku'])].copy()
reviewWithProductDF = nonAnonymousReviewDF[nonAnonymousReviewDF['sku'].isin(productDF['sku'])].copy()

In [5]:
print('shape of product df : {}'.format(productWithReviewDF.shape))
print('shape of review df : {}'.format(reviewWithProductDF.shape))

shape of product df : (7392, 4)
shape of review df : (76080, 6)


In [6]:
# drop duplicate
reviewWithProductDF = reviewWithProductDF[~reviewWithProductDF.duplicated(['sku', 'user_fk'])]
print('shape of review after drop dup : {}'.format(reviewWithProductDF.shape))

shape of review after drop dup : (75914, 6)


In [7]:
reviewWithProductCoreDF = reviewWithProductDF.drop(['comment', 'user'], axis=1)
reviewWithProductCoreDF.sample(5)

,rating,sku,time,user_fk
105058,5,SA862ELAA1JXU6ANTH,2016-09-21 14:21:15,760847
34264,4,NA554ELAA6U6M8ANTH,2017-07-06 05:53:02,6234414
103935,4,NE929HLAA5GZIHANTH,2017-04-27 15:23:00,1129503
80289,4,UN355ELAA63VOZANTH,2017-05-27 19:20:25,5436682
37932,5,NV539ELAAKN71MANTH,2017-08-14 17:30:59,11010942


In [8]:
productWithReviewCoreDF = productWithReviewDF.drop(['url'], axis=1)
productWithReviewCoreDF.sample(5)

,name,price,sku
986,Jaguar กล่องอาหารสเตนเลส ทรงรี 18 ซม. แพ็ค 2 ก...,604.0,JA696HLALKW7ANTH
4315,Kuhong Casual Men Letter Print Hip Hop Fashion...,323.0,KU876FAAAI82IZANTH
7559,Wonderful กระเป๋าเครื่องสำอางค์ รุ่น B001 -สีดำ,88.0,WO848HBAADWRW1ANTH
10800,EsoGoal (สีน้ำเงิน+สีดำ) หูฟังบลูทูธ Earbud เว...,340.0,UN355ELAACG722ANTH
10732,RELUX ตู้ลิ้นชักอเนกประสงค์ 10 ลิ้นชัก ใส่ปากก...,1290.0,RE740HLAF9Q3ANTH


In [9]:
# select only product with review count > 2
productWithManyReviews = reviewWithProductCoreDF[['sku', 'user_fk']].groupby('sku').count()
productWithManyReviews = productWithManyReviews[productWithManyReviews['user_fk'] >= 2].index.tolist()

# filter core DF
finalProductDF = productWithReviewCoreDF[productWithReviewCoreDF['sku'].isin(productWithManyReviews)]
finalReviewDF = reviewWithProductCoreDF[reviewWithProductCoreDF['sku'].isin(productWithManyReviews)]

In [10]:
# export
finalReviewDF.to_csv('../data/lazada/clean/review.csv', index=False, encoding='utf-8')
finalProductDF.to_csv('../data/lazada/clean/product.csv', index=False, encoding='utf-8')